In [1]:

import sys
sys.path.append('/home/techt/Desktop/a4s')


### Step 1: Initial Data Loading

In [2]:
from utils.preprocess import *
# 1. Identify the paths to the soft file and the matrix file
cohort_dir = '/media/techt/DATA/GEO/Heart_rate/GSE34788'
soft_file, matrix_file = geo_get_relevant_filepaths(cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Genomic signatures of a global fitness index in a multi-ethnic cohort of women"
!Series_summary	"The rates of obesity and sedentary lifestyle are on a dramatic incline, with associated detrimental health effects among women in particular. Although exercise prescriptions are useful for overcoming these problems, success can be hampered by differential responsiveness among individuals in cardiovascular fitness indices (i.e., improvements in strength, lipids, VO2max). Genetic factors appear to play an important role in determining this inter-individual variation in responsiveness.  We performed microarray analyses on mRNA in whole blood from 60 sedentary women from a multi-ethnic cohort who underwent 12 weeks of exercise, to identify gene subsets that were differentially expressed between individuals who experienced the greatest and least improvements in fitness based upon a composite fitness score index. We identified 43 transcripts in 39 unique gen

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
is_gene_available = True
trait_row = 6
age_row = None
gender_row = 1

def convert_trait(value):
    value = value.split(':')[-1].strip()
    if value == 'Low responders':
        return 0
    elif value == 'High responders':
        return 1
    else:
        return None

def convert_age(value):
    return None  # Age data is not available

def convert_gender(value):
    value = value.split(':')[-1].strip().lower()
    if value == 'female':
        return 0
    elif value == 'male':
        return 1
    else:
        return None

save_cohort_info('GSE34788', './preprocessed/Heart_rate/cohort_info.json', is_gene_available, trait_row is not None)

selected_clinical_data = geo_select_clinical_features(clinical_data, 'Heart_rate', trait_row, convert_trait, age_row, convert_age, gender_row, convert_gender)
csv_path = './preprocessed/Heart_rate/trait_data/GSE34788.csv'
selected_clinical_data.to_csv(csv_path)
print(preview_df(selected_clinical_data))


{'GSM855302': [0, 0], 'GSM855303': [0, 0], 'GSM855304': [0, 0], 'GSM855305': [0, 0], 'GSM855306': [0, 0], 'GSM855307': [0, 0], 'GSM855308': [1, 0], 'GSM855309': [1, 0], 'GSM855310': [0, 0], 'GSM855311': [0, 0], 'GSM855312': [0, 0], 'GSM855313': [0, 0], 'GSM855314': [0, 0], 'GSM855315': [0, 0], 'GSM855316': [1, 0], 'GSM855317': [1, 0], 'GSM855318': [0, 0], 'GSM855319': [0, 0], 'GSM855320': [0, 0], 'GSM855321': [0, 0], 'GSM855322': [0, 0], 'GSM855323': [0, 0], 'GSM855324': [0, 0], 'GSM855325': [0, 0], 'GSM855326': [0, 0], 'GSM855327': [0, 0], 'GSM855328': [0, 0], 'GSM855329': [0, 0], 'GSM855330': [0, 0], 'GSM855331': [0, 0], 'GSM855332': [0, 0], 'GSM855333': [0, 0], 'GSM855334': [0, 0], 'GSM855335': [0, 0], 'GSM855336': [0, 0], 'GSM855337': [0, 0], 'GSM855338': [0, 0], 'GSM855339': [0, 0], 'GSM855340': [0, 0], 'GSM855341': [0, 0], 'GSM855342': [0, 0], 'GSM855343': [0, 0], 'GSM855344': [1, 0], 'GSM855345': [1, 0], 'GSM855346': [0, 0], 'GSM855347': [0, 0], 'GSM855348': [1, 0], 'GSM855349':

### Step 3: Gene Data Extraction

In [4]:
# 1. Use the get_genetic_data function from the library to get the gene_data from the matrix_file previously defined.
gene_data = get_genetic_data(matrix_file)

# 2. Print the first 20 row ids for the following step.
print(gene_data.index[:20])


Index(['7892501', '7892502', '7892503', '7892504', '7892505', '7892506',
       '7892507', '7892508', '7892509', '7892510', '7892511', '7892512',
       '7892513', '7892514', '7892515', '7892516', '7892517', '7892518',
       '7892519', '7892520'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
requires_gene_mapping = True


### Step 5: Gene Annotation (Conditional)

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the soft file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))
    

# Configurations
identifier_key = 'ID'
gene_symbol_key = 'GB_LIST'


Gene annotation preview:
{'ID': ['7896736', '7896738', '7896740', '7896742', '7896744'], 'GB_LIST': [nan, nan, 'NM_001004195,NM_001005240,NM_001005484,BC136848,BC136867,BC136907,BC136908', 'NR_024437,XM_006711854,XM_006726377,XR_430662,AK298283,AL137655,BC032332,BC118988,BC122537,BC131690,NM_207366,AK301928,BC071667', 'NM_001005221,NM_001005224,NM_001005277,NM_001005504,BC137547,BC137568'], 'SPOT_ID': ['chr1:53049-54936', 'chr1:63015-63887', 'chr1:69091-70008', 'chr1:334129-334296', 'chr1:367659-368597'], 'seqname': ['chr1', 'chr1', 'chr1', 'chr1', 'chr1'], 'RANGE_GB': ['NC_000001.10', 'NC_000001.10', 'NC_000001.10', 'NC_000001.10', 'NC_000001.10'], 'RANGE_STRAND': ['+', '+', '+', '+', '+'], 'RANGE_START': ['53049', '63015', '69091', '334129', '367659'], 'RANGE_STOP': ['54936', '63887', '70008', '334296', '368597'], 'total_probes': [7.0, 31.0, 24.0, 6.0, 36.0], 'gene_assignment': ['---', 'ENST00000328113 // OR4G2P // olfactory receptor, family 4, subfamily G, member 2 pseudogene // ---

### Step 6: Gene Identifier Mapping

In [7]:
# 1. Get the dataframe storing the mapping between probe IDs and genes using the 'get_gene_mapping' function from the library.
mapping_df = get_gene_mapping(gene_annotation, identifier_key, gene_symbol_key)

# 2. Apply the mapping with the 'apply_gene_mapping' function from the library, and name the resulting gene expression dataframe "gene_data".
gene_data = apply_gene_mapping(gene_data, mapping_df)
